Not Working on it...

In [ ]:
#files to be imported
import numpy as np
from matplotlib import pyplot as plt
#import camb
#from camb import model, initialpower
import tensorflow as tf
import os ,sys, platform
import cosmopower as cp
import time
from tqdm import tqdm

In [2]:
el = np.loadtxt('planks/COM_PowerSpect_CMB-TT-full_R3.01.txt',usecols = 0, skiprows=1,dtype =float)
el_te = np.loadtxt('planks/COM_PowerSpect_CMB-TE-full_R3.01.txt',usecols = 0, skiprows=1,dtype =float)
Dl_TT = np.loadtxt('planks/COM_PowerSpect_CMB-TT-full_R3.01.txt',usecols = 1, skiprows=1,dtype =float)
Dl_TT_err_up = np.loadtxt('planks/COM_PowerSpect_CMB-TT-full_R3.01.txt',usecols = 3, skiprows=1,dtype =float)
Dl_TT_err_bot = np.loadtxt('planks/COM_PowerSpect_CMB-TT-full_R3.01.txt',usecols = 2, skiprows=1,dtype =float)
Dl_EE = np.loadtxt('planks/COM_PowerSpect_CMB-EE-full_R3.01.txt',usecols = 1, skiprows=1,dtype =float)
Dl_EE_err_up = np.loadtxt('planks/COM_PowerSpect_CMB-EE-full_R3.01.txt',usecols = 3, skiprows=1,dtype =float)
Dl_EE_err_bot = np.loadtxt('planks/COM_PowerSpect_CMB-EE-full_R3.01.txt',usecols = 2, skiprows=1,dtype =float)
Dl_TE = np.loadtxt('planks/COM_PowerSpect_CMB-TE-full_R3.01.txt',usecols = 1, skiprows=1,dtype =float)
Dl_TE_err_up = np.loadtxt('planks/COM_PowerSpect_CMB-TE-full_R3.01.txt',usecols = 3, skiprows=1,dtype =float)
Dl_TE_err_bot = np.loadtxt('planks/COM_PowerSpect_CMB-TE-full_R3.01.txt',usecols = 2, skiprows=1,dtype =float)
#ells
dl_fac = el*(el+1)/(2*np.pi)
dl_fac_te = el_te*(el_te+1)/(2*np.pi)
#COSMOPOWER
tt_emu = cp.cosmopower_NN(restore=True,restore_filename='COSMO/cmb_TT_NN')
ee_emu = cp.cosmopower_NN(restore=True,restore_filename='COSMO/cmb_EE_NN')
te_emu = cp.cosmopower_PCAplusNN(restore=True,restore_filename='COSMO/cmb_TE_PCAplusNN')

In [4]:
def Cosmo(xe,ze): # function for Cosmopower
    if xe == 'TT':
        tt_spectra = tt_emu.ten_to_predictions_np(ze)
        CO = tt_spectra[0]*1e12 * 2.73**2. #K^2 to uK^2
    elif xe == 'EE':
        ee_spectra = ee_emu.ten_to_predictions_np(ze)
        CA = ee_spectra[0]*1e12 * 2.73**2. #K^2 to uK^2
        CO = CA[:1995] #since planks data as only 1995 values 
    elif xe == 'TE':
        te_spectra = te_emu.predictions_np(ze)
        CA = te_spectra[0]*1e12 * 2.73**2. #K^2 to uK^2
        CO = CA[:1995] #since planks data as only 1995 values 
    else:
        Print('Check Input Range or Data')
    return CO

def chi2(tre,teu,trb,pred): #chi^2 model
    #c = ((tre - pred)**2)/((pred)**2)
    c = ((tre - pred)**2)/((teu + trb)**2) # (Real - Model)^2/(Error Up + Error Bottom)^2
    chi = np.sum(c)
    return chi

def time_hr(ms):
    seconds = int(ms/100)
    minutes = int(seconds/60)
    hours = int(minutes/60)
    days = int(hours/24)
    
    sm = int(ms - seconds*100)
    s = seconds - minutes*60 
    m = minutes - hours*60
    h = hours - days*24
    
    t = [sm,s,m,h,days]
    return t
#comparing cosmopower with planks using chi2 methode
def comparison(rollno,omb,omc,h,ta,ns,lnas):
    #params for cosmopower
    params = {'omega_b': [omb],'omega_cdm': [omc],'h': [h/100],'tau_reio': [ta],'n_s': [ns],'ln10^{10}A_s': [lnas],}
    pred_TT = Cosmo('TT',params)
    pred_TE = Cosmo('TE',params)
    pred_EE = Cosmo('EE',params)
    chi_TT = chi2(Dl_TT,Dl_TT_err_up,Dl_TT_err_bot,pred_TT)
    chi_TE = chi2(Dl_TE,Dl_TE_err_up,Dl_TE_err_bot,pred_TE)
    chi_EE = chi2(Dl_EE,Dl_EE_err_up,Dl_EE_err_bot,pred_EE)
    avg = np.average([chi_TT,chi_TE,chi_EE])
    values = [rollno,avg,chi_TT,chi_TE,chi_EE]
    return(values)

In [5]:
#comparing cosmopower with planks using chi2 methode
def comparison(rollno,omb,omc,h,ta,ns,lnas):
    #params for cosmopower
    params = {'omega_b': [omb],'omega_cdm': [omc],'h': [h/100],'tau_reio': [ta],'n_s': [ns],'ln10^{10}A_s': [lnas],}
    pred_TT = Cosmo('TT',params)
    pred_TE = Cosmo('TE',params)
    pred_EE = Cosmo('EE',params)
    chi_TT = chi2(Dl_TT,Dl_TT_err_up,Dl_TT_err_bot,pred_TT)
    chi_TE = chi2(Dl_TE,Dl_TE_err_up,Dl_TE_err_bot,pred_TE)
    chi_EE = chi2(Dl_EE,Dl_EE_err_up,Dl_EE_err_bot,pred_EE)
    avg = np.average([chi_TT,chi_TE,chi_EE])
    values = [rollno,avg,chi_TT,chi_TE,chi_EE]
    return(values)

In [3]:
#setting parmeter array for calculation
par = []
par1 = np.arange(0.0215,0.0225,0.001)  #omega_b 0.02237
par2 = np.arange(0.119,0.122,0.001)  #omega_cdm 0.1200
par3 = np.arange(67.3,67.4,0.05)  #h 67.36
par4 = np.arange(0.053,0.06,0.01)  #taurio 0.0544
par5 = np.arange(0.93,0.966,0.005)  #n_s 0.945
par6 = np.arange(3.61,3.64,0.005)  #ln10^{10}A_s 3.629
incrementor = 0
for a in par1:
    for b in par2:
        for c in par3:
            for d in par4:
                for e in par5:
                    for f in par6:
                        incrementor += 1
                        par_samples = [incrementor,a,b,c,d,e,f]
                        par.append(par_samples)

print(len(par))

1344


In [6]:
executed = [[],[],[],[],[]]
start_time = time.time()
for i in par:
    vals_calc = comparison(i[0],i[1],i[2],i[3],i[4],i[5],i[6])
    executed[0].append(vals_calc[0])
    executed[1].append(vals_calc[1])
    executed[2].append(vals_calc[2])
    executed[3].append(vals_calc[3])
    executed[4].append(vals_calc[4])
    

lows_rn = np.argmin(executed[1])

end_time = time.time()
tim = time_hr(end_time - start_time)
print(lows_rn)
print( 'Executed in :',tim[4],'d',tim[3],'H',tim[2] ,'M',tim[1],'s',tim[0],'Ms')

1231
Executed in : 0 d 0 H 0 M 1 s 62 Ms


In [13]:
lows_avg = np.argmin(executed[1])
lows_TT = np.argmin(executed[2])
lows_EE = np.argmin(executed[4])
lows_TE = np.argmin(executed[3])
ar = np.array([['roll no','omega_b' , 'omega_cdm' , 'h' , 'tau_reio' , 'n_s ',' ln10^{10}A_s'],par[lows_avg],par[lows_TT],par[lows_EE],par[lows_TE]])
print(ar)

[['roll no' 'omega_b' 'omega_cdm' 'h' 'tau_reio' 'n_s ' ' ln10^{10}A_s']
 ['1232' '0.0225' '0.122' '67.3' '0.053' '0.9650000000000001'
  '3.6399999999999992']
 ['1232' '0.0225' '0.122' '67.3' '0.053' '0.9650000000000001'
  '3.6399999999999992']
 ['119' '0.0215' '0.119' '67.39999999999999' '0.053' '0.93'
  '3.6399999999999992']
 ['168' '0.0215' '0.119' '67.39999999999999' '0.053' '0.9650000000000001'
  '3.6399999999999992']]
